In [2]:
import os
import pandas as pd
import unidecode
from datetime import time
import numpy as np


# PROCESSING ET CLEANING ✅

In [3]:
# Charger les fichiers CSV
def load_csv_files(csv_folder, file_prefix="EXP_", sep=';'):
    dataframes = {}
    files = [f for f in os.listdir(csv_folder) if f.startswith(file_prefix) and f.endswith(".csv")]
    for file in files:
        df_name = file.replace(file_prefix, "").replace(".csv", "").capitalize()
        df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)
        dataframes[df_name] = df
    return dataframes

# Sélection et renommage des colonnes
def select_and_rename_columns(dataframes, config):
    for name, settings in config.items():
        if name in dataframes:
            df = dataframes[name]
            if 'keep' in settings:
                df = df[settings['keep']]
            if 'rename' in settings:
                df.rename(columns=settings['rename'], inplace=True)
            if 'drop_pattern' in settings:
                drop_cols = df.filter(regex=settings['drop_pattern']).columns
                df.drop(columns=drop_cols, inplace=True)
            dataframes[name] = df

# Nettoyage des noms de colonnes
def clean_column_names(dataframes):
    for name, df in dataframes.items():
        df.columns = [unidecode.unidecode(col).lower().replace(" ", "_").replace("-", "_") for col in df.columns]
        dataframes[name] = df

# Conversion des types avec gestion spécifique pour la colonne `sur`
def convert_column_types(dataframes, column_types):
    for name, df in dataframes.items():
        for col in df.columns:
            # Conversion des colonnes string
            if col in column_types.get('string', []):
                df[col] = df[col].astype(str)

            # Conversion spécifique pour la colonne `sur`
            elif col == 'sur':
                # Extraire la valeur après '/' et convertir en entier
                df[col] = pd.to_numeric(df[col].astype(str).str.replace('/', ''), errors='coerce').astype('Int64')

            # Conversion des colonnes float
            elif col in column_types.get('float', []):
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

            # Conversion des colonnes int
            elif col in column_types.get('int', []) and col != 'sur':  # Exclure `sur` pour éviter un double traitement
                # Convertir d'abord en float pour traiter les valeurs décimales et les `NaN`
                temp_col = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

                # Vérifier si toutes les valeurs non `NaN` sont des entiers
                if (temp_col.dropna() % 1 == 0).all():
                    # Convertir en `Int64` en utilisant `.fillna(np.nan).astype('Int64')`
                    df[col] = temp_col.fillna(np.nan).astype('Int64')
                else:
                    # Si des valeurs décimales existent, laisser en `float`
                    df[col] = temp_col

            # Conversion des colonnes date
            elif col in column_types.get('date', []):
                df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

            # Conversion des colonnes time
            elif col in column_types.get('time', []):
                df[col] = pd.to_datetime(df[col].astype(str).str.replace('h', ':'), format='%H:%M', errors='coerce').dt.time

            # Conversion des colonnes category
            elif col in column_types.get('category', []):
                df[col] = df[col].astype('category')

        # Mettre à jour le DataFrame dans le dictionnaire
        dataframes[name] = df

def anonymize_students(dataframes, eleve_df, ident_column='ident'):
    """
    Anonymise les informations des élèves dans les DataFrames spécifiés.

    Parameters:
        dataframes (dict): Dictionnaire de DataFrames à anonymiser.
        eleve_df (DataFrame): DataFrame contenant les informations des élèves avec l'identifiant.
        ident_column (str): Nom de la colonne d'identifiant dans eleve_df (par défaut 'ident').
    """
    # Créer le mapping pour les élèves
    eleve_mapping = {f"{row['nom']} {row['prenom']}": row[ident_column] for _, row in eleve_df.iterrows()}

    # Liste des DataFrames à anonymiser pour les élèves
    student_dfs = ['Punition', 'Notes','Notesdevoir_t1','Notesdevoir_t2', 'Notesdevoir_t3', 'Absenceseleves', 'Retards', 'Appreciationprofesseurs',
                   'Passagesinfirmerie', 'Appreciationduconseil', 'Sanction', 'Notes','Absencerepas']

    for name in student_dfs:
        if name in dataframes:
            df = dataframes[name]
            if 'nom' in df.columns and 'prenom' in df.columns:
                df['nom_prenom'] = df['nom'] + ' ' + df['prenom']
                df['eleve_id'] = df['nom_prenom'].map(eleve_mapping)
                df.drop(columns=['nom', 'prenom', 'nom_prenom'], inplace=True)
                dataframes[name] = df

def anonymize_professors(dataframes, professor_df, ident_column='ident'):
    """
    Anonymise les informations des professeurs dans les DataFrames spécifiés.

    Parameters:
        dataframes (dict): Dictionnaire de DataFrames à anonymiser.
        professor_df (DataFrame): DataFrame contenant les informations des professeurs avec l'identifiant.
        ident_column (str): Nom de la colonne d'identifiant dans professor_df (par défaut 'ident').
    """
    # Créer le mapping pour les professeurs
    professor_df['nom_complet'] = professor_df['nom'] + ' ' + professor_df['prenom']
    prof_mapping = {row['nom_complet']: row[ident_column] for _, row in professor_df.iterrows()}

    # Liste des DataFrames à anonymiser pour les professeurs
    professor_dfs = ['Notesdevoir_t1','Notesdevoir_t2', 'Notesdevoir_t3', 'Appreciationprofesseurs', 'Service', 'Notes']

    for name in professor_dfs:
        if name in dataframes and 'profs' in dataframes[name].columns:
            df = dataframes[name]
            df['profs_standard'] = df['profs'].str.replace(r'^M\.|Mme\s+', '', regex=True).str.strip()
            df['prof_id'] = df['profs_standard'].map(prof_mapping)
            df.drop(columns=['profs', 'profs_standard'], inplace=True)
            dataframes[name] = df


In [4]:
# Configurer la sélection et le renommage des colonnes
columns_config = {
    'Punition': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PUNITION', 'DATE', 'MOTIF', 'HEURE']
    },
    'Notesdevoir_t1': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },
    'Notesdevoir_t2': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },
    'Notesdevoir_t3': {
        'keep': ['CLASSES', 'NOM', 'PRENOM', 'PROFS', 'MATIERE', 'DATE', 'P1', 'SUR', 'COEFF', 'NOTE'],
        'rename': {'P1': 'TRIMESTRE'}

    },

    'Professeur': {
        'keep': ['IDENT', 'CIVILITE', 'NOM', 'PRENOM']
    },
    'Absenceseleves': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'DATE DEBUT', 'DEMI JOUR'],
        'rename': {'DATE DEBUT': 'DATE', 'DEMI JOUR': 'NB_DEMI_JOURNEE'}
    },
    'Absencerepas': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE']
    },
    'Retards': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'MOTIF', 'REGLE', 'DATE', 'HEURE', 'DUREE']
    },
    'Eleve': {
        'keep': ['IDENT', 'NOM', 'PRENOM', 'DATE NAISS', 'CLASSES', 'SEXE'],
        'rename': {'DATE NAISS': 'DATE_NAISS'}
    },
    'Service': {
        'keep': ['MATIERE', 'CLASSES', 'PROFS', 'NB DEVOIRS', 'COEFF'],
        'rename': {'NB DEVOIRS': 'NB_DEVOIRS'}
    },
    'Appreciationprofesseurs': {
        'keep': ['NOM', 'PRENOM', 'MATIERE', 'CLASSE/GRPE', 'PROFS', 'PERIODE', 'APPR.A'],
        'rename': {'CLASSE/GRPE': 'CLASSE', 'APPR.A': 'APPR'}
    },
    'Passagesinfirmerie': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'H DEBUT', 'DATE', 'DUREE'],
        'rename': {'H DEBUT': 'HEURE'}
    },
    'Appreciationduconseil': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'PERIODE', 'APPR.C'],
        'rename': {'APPR.C': 'APPR'}
    },
    'Sanction': {
        'keep': ['NOM', 'PRENOM', 'CLASSES', 'DATE', 'SANCTION', 'MOTIF']
    },
    'Notes': {
        'keep': ['PROFS', 'MATIERE', 'CLASSE/GRPE', 'NOM ELEVE', 'PRENOM ELEVE', 'MOY_ELEVE'] +
                [f'Devoir{i} - Note' for i in range(1, 25)] +
                [f'Devoir{i} - Coeff' for i in range(1, 25)] +
                [f'Devoir{i} - Date' for i in range(1, 25)] +
                [f'Devoir{i} - Période Notation' for i in range(1, 25)],
                'rename': {'NOM ELEVE': 'NOM', 'PRENOM ELEVE':'PRENOM'},
        'drop_pattern': 'Devoir\d+ - Facultatif'
    }
}

# Configurer les types de colonnes
column_types = {
    'string': ['nom', 'prenom', 'classes','profs', 'civilite', 'motif', 'punition', 'regle'],
    'float': ['note', 'moyenne'],
    'int': ['nb_demi_journee', 'nb_devoirs','coeff', 'sur'],
    'date': ['date', 'date_naiss', 'periode'],
    'time': ['heure', 'duree'],
    'category': ['classe', 'trimestre', 'matiere', 'sexe']
}


# Application
csv_folder_path = "../../../raw_data/Daudet/2023-2024/Pronote/"

#extraction des données
data_pronote = load_csv_files(csv_folder=csv_folder_path)

#selection des colonnes
select_and_rename_columns(data_pronote, columns_config)

#cleaning des noms de colonnes
clean_column_names(data_pronote)

#cleaning des types de colonnes
convert_column_types(data_pronote, column_types)

# Appliquer l'anonymisation des élèves
anonymize_students(data_pronote, data_pronote['Eleve'])

# Appliquer l'anonymisation des professeurs
anonymize_professors(data_pronote, data_pronote['Professeur'])

/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_80372/2906436130.py:7: DtypeWarning: Columns (81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(csv_folder, file), sep=sep)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_80372/2906436130.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=settings['rename'], inplace=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_80372/2906436130.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

#### Tous les dataframes sont crées, structurés, nettoyées, anonymisés ✅

# DF_NOTES

In [5]:
# Ajouter une colonne 'trimestre' pour chaque DataFrame avant de les concaténer
data_pronote['Notesdevoir_t1']['trimestre'] = 'Trimestre 1'
data_pronote['Notesdevoir_t2']['trimestre'] = 'Trimestre 2'
data_pronote['Notesdevoir_t3']['trimestre'] = 'Trimestre 3'

# Concaténer les trois DataFrames
df_notes_devoir = pd.concat([
    data_pronote['Notesdevoir_t1'],
    data_pronote['Notesdevoir_t2'],
    data_pronote['Notesdevoir_t3']
], ignore_index=True)

# Afficher un aperçu du DataFrame concaténé
df_notes_devoir.head()

,classes,matiere,date,trimestre,sur,coeff,note,eleve_id,prof_id
0,1A,FRANÇAIS,2023-09-21,Trimestre 1,20,1.0,13.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,0-qkoCBCeu1-A2WvQZEecuK98nvosoTdzUTdiI6lwDo
1,1A,ENSEIGNEMENT SCIENTIFIQUE (PC),2023-09-25,Trimestre 1,20,0.5,7.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,zalvxQljlC_iyENJQK4qClMhXqp3ZogCFPq9JR9Yalw
2,1A,SPE MATHS,2023-09-26,Trimestre 1,20,1.0,8.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,JumFXm1J2Jm1zAFbJ-KKVcMFxl2mfiu882gBggAozMg
3,1A,HISTOIRE GEOGRAPHIE,2023-09-27,Trimestre 1,10,1.0,5.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,MkTDEy0fzub_lzFNRBxLLG0sLd5mzSpMxX3wx8l6Guo
4,1A,ARABE LVA,2023-10-04,Trimestre 1,20,2.0,12.0,x98XOvwoNyehe5aGZFC4QGm8ZPyM6Sof0LJwXpfNP-c,t8jRAdLt_P0Ai2D2pyBamlLBFMnN_k9Od6ycTUB98oA


In [9]:
import os

# Chemin du dossier d'exportation
export_folder = "exported_data"
os.makedirs(export_folder, exist_ok=True)  # Crée le dossier s'il n'existe pas

# Fonction pour exporter les DataFrames dans le dossier spécifié
def export_dataframes(dataframes, folder_path):
    """
    Exporte chaque DataFrame dans un fichier CSV distinct dans le dossier spécifié.

    Parameters:
        dataframes (dict): Dictionnaire contenant les DataFrames à exporter.
        folder_path (str): Chemin du dossier d'exportation.
    """
    for name, df in dataframes.items():
        file_path = os.path.join(folder_path, f"{name}.csv")  # Chemin complet du fichier
        df.to_csv(file_path, index=False, sep=';')  # Export en CSV avec ';' comme séparateur
        print(f"DataFrame '{name}' exporté vers {file_path}")

# Ajout de tous les DataFrames à exporter, y compris ceux dans `data_pronote`
dataframes_to_export = {
    'df_notes_devoir': df_notes_devoir,                # DataFrame avec les notes par devoir
    **data_pronote  # Inclut tous les DataFrames contenus dans data_pronote
}

# Exporter tous les DataFrames
export_dataframes(dataframes_to_export, export_folder)


DataFrame 'df_notes_devoir' exporté vers exported_data/df_notes_devoir.csv
DataFrame 'Notesdevoir_t3' exporté vers exported_data/Notesdevoir_t3.csv
DataFrame 'Punition' exporté vers exported_data/Punition.csv
DataFrame 'Notesdevoir_t2' exporté vers exported_data/Notesdevoir_t2.csv
DataFrame 'Notesdevoir' exporté vers exported_data/Notesdevoir.csv
DataFrame 'Notesdevoir_t1' exporté vers exported_data/Notesdevoir_t1.csv
DataFrame 'Professeur' exporté vers exported_data/Professeur.csv
DataFrame 'Absenceseleves' exporté vers exported_data/Absenceseleves.csv
DataFrame 'Absencerepas' exporté vers exported_data/Absencerepas.csv
DataFrame 'Retards' exporté vers exported_data/Retards.csv
DataFrame 'Eleve' exporté vers exported_data/Eleve.csv
DataFrame 'Service' exporté vers exported_data/Service.csv
DataFrame 'Appreciationprofesseurs' exporté vers exported_data/Appreciationprofesseurs.csv
DataFrame 'Passagesinfirmerie' exporté vers exported_data/Passagesinfirmerie.csv
DataFrame 'Appreciationduc